# Multithreading

## Overview

* **What are threads?**

* **How many threads are there?**

* **Where are the threads running?**

* **How do I use the threads?**

## What are threads?
Threads are **execution units within a process** that can run simultaneously and **share memory** (heap).

<br>
<img src="./imgs/stack_heap_threads.png" width=450px>
<br>

## How many threads are there?

By default, Julia starts with a single *user thread*. We must tell it explicitly to start multiple user threads.

* Environment variable: `export JULIA_NUM_THREADS=8`

* Command line argument: `julia -t 8` or `julia --threads 8`

**It is currently not really possible to change the number of threads at runtime!**

In [1]:
Threads.nthreads()

8

## Where are the threads running?

[ThreadPinning.jl](https://github.com/carstenbauer/ThreadPinning.jl) is the best tool for visualizing and controlling thread placement in Julia. (Disclaimer: I'm the main author 😉)

In [2]:
using ThreadPinning

threadinfo()


System: 128 cores (2-way SMT), 2 sockets, 8 NUMA domains

| 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,
  16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
  32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,
  48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
  128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,
  144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,
  160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,
  176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191 |
| 64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,
  80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,
  96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,
  112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,
  192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,
  208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,
  224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,
  240,241,242,243,244,245,246,247,248,249,250

### Pinning threads (i.e. controling where they are running)

#### Why?

* To avoid double occupancy of CPU cores.

* To reduce noise in benchmarks.

* To address the complexity of the system topology, e.g. to use specific/all memory domains (NUMA).

* ...

#### How?

`pinthreads(strategy)`
* `:cputhreads` pin to CPU threads (incl. "hypterthreads") one after another
* `:cores:` pin to CPU cores one after another
* `:numa:` alternate between NUMA domains (round-robin)
* `:sockets:` alternate between sockets (round-robin)
* `:affinitymask`: pin according to an external affinity mask (e.g. set by SLURM)

(More? See my talk at JuliaCon2023 @ MIT: https://youtu.be/6Whc9XtlCC0)

In [13]:
pinthreads(:cores) # try :cores or :sockets or :random
threadinfo()


System: 128 cores (2-way SMT), 2 sockets, 8 NUMA domains

| 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,
  16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
  32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,
  48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
  128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,
  144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,
  160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,
  176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191 |
| 64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,
  80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,
  96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,
  112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,
  192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,
  208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,
  224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,
  240,241,242,243,244,245,246,247,248,249,250

In [14]:
pinthreads(:numa)
threadinfo(; groupby=:numa)


System: 128 cores (2-way SMT), 2 sockets, 8 NUMA domains

| 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,
  128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143 |
| 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
  144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159 |
| 32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,
  160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175 |
| 48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
  176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191 |
| 64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,
  192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207 |
| 80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,
  208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223 |
| 96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,
  224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239 |
| 112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,
  240,241,242,243,244,245,246,247,248,2

#### Memory domains (NUMA)

NUMA = **n**on-**u**niform **m**emory **a**ccess

One (of two) AMD Milan CPUs in a Perlmutter node:

<img src="imgs/amd_milan_cpu_die.svg" width=800px>

**Image source:** [AMD, High Performance Computing (HPC) Tuning Guide for AMD EPYCTM 7003 Series Processors](https://www.amd.com/system/files/documents/high-performance-computing-tuning-guide-amd-epyc7003-series-processors.pdf)

In [ ]:
# Other useful options for querying system information

# using CpuId
# cpuinfo()

# using Hwloc
# topology_graphical()

## How do I use the threads?

### Task-based multithreading

In traditional HPC, one typically tells each thread what to do. **("One thinks about threads")**

Julia implements **task-based multithreading**. In this paradigm, a task - e.g. a computational piece of a code - is marked for **parallel** execution on **any** of the available Julia threads. Julia's **dynamic scheduler** will automatically put the task on one of the threads and trigger the execution of the task on said thread.

<br>
<img src="./imgs/tasks_threads_cores.svg" width=750px>
</br>

Generally speaking, the user should **think about tasks and not threads**.
* The scheduler is controlling on which thread a task will eventually run.
* It might even dynamically [migrate tasks](https://docs.julialang.org/en/v1/manual/multi-threading/#man-task-migration) between threads.

**Advantages:**
* high-level abstraction
* nestability / composability (especially important for libraries)

**Disadvantages:**
* scheduling overhead
* uncertain and potentially suboptimal task → thread assignment
  * **can get in the way when performance engineering** because
    * scheduler has limited information (e.g. about the system topology)
    * profiling tools often don't know anything about tasks but monitor threads (or even CPU-cores) instead (e.g. LIKWID).

### `@threads for ... in ...`

* **Splits up the iteration space into `nthreads()` contiguous chunks**

* Creates a task for each of them and hands them off to the dynamic scheduler (essentially `@spawn`).

In [15]:
using Base.Threads: @threads, threadid, nthreads
using ThreadPinning: taskid # for pedagogical purposes only

In [16]:
# creates nthreads() many tasks

@threads for i in 1:2*nthreads()
    println("Task ", ThreadPinning.taskid(), " is running iteration ", i, " on thread ", threadid())
end

Task 14861510704624281442 is running iteration 3 on thread 6
Task 14175704995940435965 is running iteration 9 on thread 8
Task 14861510704624281442 is running iteration 4 on thread 6
Task 14175704995940435965 is running iteration 10 on thread 8
Task 2638481533447028374 is running iteration 7 on thread 5
Task 11763296367304083218 is running iteration 5 on thread 7
Task 2638481533447028374 is running iteration 8 on thread 5
Task 11910625230368068474 is running iteration 1 on thread 1
Task 11910625230368068474 is running iteration 2 on thread 8
Task 1984371753562007792 is running iteration 11 on thread 4
Task 15990662284487850768 is running iteration 13 on thread 2
Task 15990662284487850768 is running iteration 14 on thread 2
Task 1984371753562007792 is running iteration 12 on thread 4
Task 11763296367304083218 is running iteration 6 on thread 7
Task 17984801083959781576 is running iteration 15 on thread 3
Task 17984801083959781576 is running iteration 16 on thread 3


#### Static scheduling: `@threads :static for ... in ...`

* `:static` scheduling option to opt-out of Julia's dynamic scheduling

* Statically **"pins" tasks to threads**
  * task 1 → thread 1, task 2 → thread 2, and so on.

Pro
   * no task migration, i.e. **fixed task-thread mapping** 👍
   
Con
   * only little overhead 👍
   * not composable / nestable 👎

In [17]:
@threads :static for i in 1:2*nthreads()
    println("Task ", taskid(), " is running iteration ", i, " on thread ", threadid());
end

Task 1944217483586526427 is running iteration 3 on thread 2
Task 17213828643279112869 is running iteration 7 on thread 4
Task 13740810260962294991 is running iteration 11 on thread 6
Task 17565571350995400555 is running iteration 15 on thread 8
Task 13740810260962294991 is running iteration 12 on thread 6
Task 16256560978224147096 is running iteration 5 on thread 3
Task 3712476857570664011 is running iteration 9 on thread 5
Task 17565571350995400555 is running iteration 16 on thread 8
Task 4303338266125538787 is running iteration 13 on thread 7
Task 16320237495516898705 is running iteration 1 on thread 1
Task 1944217483586526427 is running iteration 4 on thread 2
Task 16320237495516898705 is running iteration 2 on thread 1
Task 17213828643279112869 is running iteration 8 on thread 4
Task 16256560978224147096 is running iteration 6 on thread 3
Task 3712476857570664011 is running iteration 10 on thread 5
Task 4303338266125538787 is running iteration 14 on thread 7


For `@threads :static`, every thread handles precisely two iterations!

## "Data pinning" (NUMA revisited)

Implicitly → **NUMA "first-touch" policy**

Explicitly → [NUMA.jl](https://github.com/JuliaPerf/NUMA.jl)

### NUMA "first-touch" policy

Data is (typically) placed in the **NUMA domain that is closest to the thread/CPU core** that is "touching" the data.

```julia
x = Vector{Float64}(undef, 10)   # allocation, no "touch" yet
rand!(x)                         # first touch == first write
```

In [18]:
pinthreads(:numa)
threadinfo(; groupby=:numa)


System: 128 cores (2-way SMT), 2 sockets, 8 NUMA domains

| 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,
  128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143 |
| 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
  144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159 |
| 32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,
  160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175 |
| 48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
  176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191 |
| 64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,
  192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207 |
| 80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,
  208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223 |
| 96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,
  224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239 |
| 112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,
  240,241,242,243,244,245,246,247,248,2

### Array initialization: serial vs parallel

**Different parts of an array can be placed in different NUMA domains!**

(Data is managed in terms of smaller units, namley memory pages.)

#### Serial

```julia
x = Vector{Float64}(undef, 100)   # allocation, no "touch" yet
rand!(x)                          # first touch == first write
```

The location of the "main" thread determines the NUMA domain of the entire array!

If we later access the data in parallel, all threads must read from the same NUMA domain → competition for the memory bus → potential bottleneck.

#### Parallel

```julia
pinthreads(:numa)                       # pin threads to different NUMA domains
x = Vector{Float64}(undef, 100)         # allocation, no "touch" yet
@threads :static for i in eachindex(x)  # parallel iteration
    x[i] = rand()                       # first touch == first write
end
```

Different threads - running in different NUMA regions - touch different parts of the array → the latter will (likely) be placed in different NUMA domains.

If we later access the data in parallel, all threads can read their part of the array from their local NUMA domain → no bottleneck.

**Crucial point: How you initialize your data influences the performance of your computational kernel(s)!**
* up to a factor of 8 performance difference possible on Perlmutter

* non-local performance effect → hard to debug